### Load database

In [90]:
### DB loader
import pandas as pd

db = pd.concat([
    pd.read_csv("../cblas_db/blas.csv", index_col=0),
    pd.read_csv("../lapacke_db/lapacke.csv", index_col=0)
])

### Function unwrapper

In [91]:
data_table_c = {
        # BLAS types
        'const int':    'int',
        'const float':  'cste_c_binary',
        'const double': 'cste_c_binary',
        'const float*': 'cste_c_binary',
        'const double*':'cste_c_binary',
        'const void*':  'cste_c_binary',
        'void*':        'c_binary',
        'float*':       'c_binary',
        'double*':      'c_binary',
        
        'const enum CBLAS_ORDER':       'int',
        'const enum CBLAS_TRANSPOSE':   'int',
        'const enum CBLAS_UPLO':        'int',
        'const enum CBLAS_DIAG':        'int',
        'const enum CBLAS_SIDE':        'int',

        # LAPACKE types
        'int':          'int',                  # Only used for matrix_layout
        'char':         'char',
        'lapack_int':   'int',
        'lapack_logical': 'int',
        'float':        'cste_c_binary',
        'double':       'cste_c_binary',
        #'lapack_complex_float':  'c_binary',
        #'lapack_complex_double': 'c_binary',

        'lapack_int*':  'c_binary',
        'lapack_logical*':  'c_binary',
        'lapack_complex_float*': 'c_binary',
        'lapack_complex_double*': 'c_binary',

        'const lapack_int*':  'cste_c_binary',
        'const lapack_logical*':       'cste_c_binary',
        'const lapack_complex_float*': 'cste_c_binary',
        'const lapack_complex_double*': 'cste_c_binary',

}
def arg_type_c(s):
    return data_table_c[s]

def arg_type_decoder_type(s):
    data_table = {
        # BLAS types
        'const int':    'e_int',
        'const float':  'e_cste_ptr',
        'const double': 'e_cste_ptr',
        'const float*': 'e_cste_ptr',
        'const double*':'e_cste_ptr',
        'const void*':  'e_cste_ptr',
        'void*':        'e_ptr',
        'float*':       'e_ptr',
        'double*':      'e_ptr',
        
        'const enum CBLAS_ORDER':       'e_layout',
        'const enum CBLAS_TRANSPOSE':   'e_transpose',
        'const enum CBLAS_UPLO':        'e_uplo',
        'const enum CBLAS_DIAG':        'e_diag',
        'const enum CBLAS_SIDE':        'e_side',

        # LAPACKE types
        'int':          'e_layout',                  # Only used for matrix_layout
        'char':         'e_char',
        'lapack_int':   'e_int',
        'lapack_logical': 'e_int',
        'float':        'e_cste_ptr',
        'double':       'e_cste_ptr',
        'lapack_complex_float':  'e_ptr',
        'lapack_complex_double': 'e_ptr',

        'lapack_int*':  'e_ptr',
        'lapack_logical*':       'e_ptr',
        'lapack_complex_float*': 'e_ptr',
        'lapack_complex_double*': 'e_ptr',

        'const lapack_int*':            'e_cste_ptr',
        'const lapack_logical*':        'e_cste_ptr',
        'const lapack_complex_float*':  'e_cste_ptr',
        'const lapack_complex_double*': 'e_cste_ptr',
    }
    return data_table[s]


def arg_unwrapper(initial_type, var_name):
    data_table = {
        # BLAS TYPES
        'const int':    lambda x: x,
        'const float':  lambda x: f'get_cste_float({x})',
        'const double': lambda x: f'get_cste_double({x})',
        'const float*': lambda x: f'get_cste_ptr({x})',
        'const double*':lambda x: f'get_cste_ptr({x})',
        'const void*':  lambda x: f'get_cste_ptr({x})',
        'void*':        lambda x: f'get_ptr({x})',
        'float*':       lambda x: f'get_ptr({x})',
        'double*':      lambda x: f'get_ptr({x})',
        
        'const enum CBLAS_ORDER':       lambda x: x,
        'const enum CBLAS_TRANSPOSE':   lambda x: x,
        'const enum CBLAS_UPLO':        lambda x: x,
        'const enum CBLAS_DIAG':        lambda x: x,
        'const enum CBLAS_SIDE':        lambda x: x,

        # LAPACKE types
        'int':          lambda x: x,                  # Only used for matrix_layout
        'char':         lambda x:x,
        'lapack_int':   lambda x: x,
        'lapack_logical': lambda x: x,
        'float':        lambda x: f'get_cste_double({x})',
        'double':       lambda x: f'get_cste_double({x})',
        'lapack_complex_float':     lambda x: f'get_ptr({x})',
        'lapack_complex_double':    lambda x: f'get_ptr({x})',

        'lapack_int*':  lambda x: f'get_ptr({x})',
        'lapack_logical*':  lambda x: f'get_ptr({x})',
        'lapack_complex_float*':  lambda x: f'get_ptr({x})',
        'lapack_complex_double*': lambda x: f'get_ptr({x})',
        
        'const lapack_int*':            lambda x: f'get_cste_ptr({x})',
        'const lapack_logical*':        lambda x: f'get_cste_ptr({x})',
        'const lapack_complex_float*':  lambda x: f'get_cste_ptr({x})',
        'const lapack_complex_double*': lambda x: f'get_cste_ptr({x})',
    }
    return data_table[initial_type](var_name)
    

In [92]:
import re

def declare_variables(variables):

    declarations = pd.concat(
        [
            pd.DataFrame(variables['arg_type'].apply(arg_type_c)),
            variables['arg_name']
        ],
        axis = 1
    ).to_string(index=False, header=False, col_space=1, justify='left').replace('\n', '; ')
    declarations = re.sub(r"\s+", " ", declarations) + ";"
    return declarations.strip()


def get_narg(variables):
    return variables['arg_pos'].max() + 1


def get_translate(variables):
    return '{' + ', '.join([t for t in variables['arg_type'].apply(arg_type_decoder_type)]) + ", e_end}, " + ', '.join(['&' + n for n in variables['arg_name']])

def get_arg_unwrapper(variables):
    return ', '.join([arg_unwrapper(t,v) for t,v in variables[['arg_type', 'arg_name']].values])


In [93]:
def gen_fct(fct_name):
    variables = db[db['fct_name']==fct_name]
    return f"""
        case {variables.iloc[0]['e_name']}: {{
            {declare_variables(variables)}
            
            if( !(error = narg == {get_narg(variables)}? 0:ERROR_N_ARG)
                && !(error = translate(env, elements, (etypes[]) {get_translate(variables)}))
            ){{
                {fct_name}({get_arg_unwrapper(variables)});
            }}
        }} break;"""

In [94]:
# Filtering
# only BLAS
#db = db[ (db['return_type'] == 'void')]
# BLAS and LAPACK with easy to use return type
#db = db[ (db['return_type'] == 'void') | (db['return_type'] == 'lapack_int')]

#lapack_to_del = db[db['arg_type'].str.contains('LAPACK')]['fct_name'].unique()
#db = db[~db['fct_name'].str.contains('|'.join(lapack_to_del))]

missing = db[~db['arg_type'].isin(data_table_c.keys())]['arg_type'].unique()
print("Missing arguments:")
print(missing)
lapacke_filtered = db.groupby('fct_name').filter(lambda x: all(x['arg_type'].isin(data_table_c.keys())))['fct_name'].unique().tolist()
db = db[(db['fct_name'].isin(lapacke_filtered)) | (db['return_type'] == 'void')]

Missing arguments:
['char*' 'LAPACK_S_SELECT2' 'LAPACK_D_SELECT2' 'LAPACK_C_SELECT1'
 'LAPACK_Z_SELECT1' 'LAPACK_S_SELECT3' 'LAPACK_D_SELECT3'
 'LAPACK_C_SELECT2' 'LAPACK_Z_SELECT2' 'lapack_complex_float'
 'lapack_complex_double']


In [95]:
functions_wrappers = ''.join(map(gen_fct, db['fct_name'].unique()))

with open("unwrappers.txt", "w") as f:
    f.write(functions_wrappers)

### Hash tables

In [96]:
def hash(string):
    hash_value = 5381

    for char in string:
        hash_value = ((hash_value << 5) + hash_value) + ord(char)

    return hash_value

In [97]:
tests = {
    'saxpy':210727551034,
    'daxpy':210709762219,
    'caxpy':210708576298,
    'zaxpy':210735852481,
}

for k,v in tests.items():
    if hash(k) != v:
        print(f"Error for {k}: expected {v}, obtained {hash(k)}")

In [98]:
hashes = "".join(map(lambda x: f"   {x} = {hash(x)},\n", db["e_name"].unique()))

with open("hashes.txt", "w") as f:
    f.write(hashes)

#### Deploy

In [99]:
def insert_in(woman, man, child):
    with open(woman, 'r') as woman_f, open(man, 'r') as man_f, open(child, 'w') as child_f:
        woman_txt = woman_f.read().split('//INSERT_HERE')
        man_txt   = man_f.read()
        child_f.write(woman_txt[0] + man_txt + woman_txt[1])

insert_in("eblas.pseudo_c", "unwrappers.txt", "../../blas/c_src/eblas.c")
insert_in("tables.pseudo_h", "hashes.txt", "../../blas/c_src/tables.h")

In [100]:
count = len(db[db['fct_name'].str.contains('LAPACKE', case=False)]['fct_name'].unique())
print(count)

798
